In [1]:
import pandas as pd

# load data
df = pd.read_csv('../data/york_data_clean.csv')

df.head()

,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,bfi52_mapped,bfi53_mapped,bfi54_mapped,bfi55_mapped,bfi56_mapped,bfi57_mapped,bfi58_mapped,bfi59_mapped,bfi60_mapped,bfi_combined
0,9/23/24 20:58,9/23/24 21:09,0,99.247.239.74,100,692,1,9/23/24 21:09,R_5d45GouOkUfD9rf,NaN,...,I am fairly polite courteous to others.,I am fairly persistent usually work until the ...,I almost never feel depressed blue.,I have little interest in abstract ideas.,I show a fair amount of enthusiasm.,I sometimes assume the best about people.,I sometimes behave irresponsibly.,I am not at all temperamental almost never get...,I am quite original often come up with new ideas.,"I am quite outgoing, sociable. I am not at all..."
1,9/23/24 21:06,9/23/24 21:16,0,70.35.217.132,100,576,1,9/23/24 21:16,R_7h2Xcu888ru6Lap,NaN,...,I am fairly polite courteous to others.,I am fairly persistent usually work until the ...,I sometimes feel depressed blue.,I have some interest in abstract ideas.,I show a fair amount of enthusiasm.,I sometimes assume the best about people.,I rarely behave irresponsibly.,I am quite temperamental often get emotional.,I am moderately original sometimes come up wit...,"I am somewhat outgoing, sociable. I am fairly ..."
2,9/23/24 20:59,9/23/24 21:17,0,70.53.73.20,100,1107,1,9/23/24 21:17,R_3pXno2C5AT8NsOE,NaN,...,I am fairly polite courteous to others.,I am very persistent almost always work until ...,I almost never feel depressed blue.,I have a lot of interest in abstract ideas.,I show a lot of enthusiasm.,I usually assume the best about people.,I almost never behave irresponsibly.,I am moderately temperamental sometimes get em...,I am very original almost always come up with ...,"I am quite outgoing, sociable. I am moderately..."
3,9/23/24 21:04,9/23/24 21:22,0,23.233.10.215,100,1080,1,9/23/24 21:22,R_76kJbfHLQrYBL4X,NaN,...,I am fairly polite courteous to others.,I am somewhat persistent sometimes work until ...,I almost always feel depressed blue.,I have a fair amount of interest in abstract i...,I show almost no enthusiasm.,I rarely assume the best about people.,I rarely behave irresponsibly.,I am quite temperamental often get emotional.,I am moderately original sometimes come up wit...,"I am quite reserved, unsociable. I am moderate..."
4,9/23/24 21:00,9/23/24 21:29,0,72.136.110.98,100,1738,1,9/23/24 21:29,R_1uFF8HZ890wuWdP,NaN,...,I am fairly polite courteous to others.,I am somewhat persistent sometimes work until ...,I almost never feel depressed blue.,I have some interest in abstract ideas.,I show some enthusiasm.,I usually assume the best about people.,I rarely behave irresponsibly.,I am moderately temperamental sometimes get em...,I am moderately original sometimes come up wit...,"I am somewhat outgoing, sociable. I am fairly ..."


# Test with a single participant

In [2]:
from openai import OpenAI
from risk_taking import get_prompt
import os

# Test with a single participant
person1 = df.iloc[0]

# pass person1['combined'] to the get_prompt function
prompt = get_prompt(person1['bfi_combined'])

# print the prompt
print(prompt)

### Personality### 
I am quite outgoing, sociable. I am not at all compassionate almost never soft-hearted. I am fairly organized. I am very relaxed handle stress very well. I have no artistic interests. I am moderately assertive. I am very respectful almost always treat others with respect. I am sometimes lazy. I stay very optimistic after experiencing a setback. I am curious about a large number of things. I often feel excited or eager. I sometimes find fault with others. I am very dependable steady. I am somewhat moody sometimes have up and down mood swings. I am very inventive almost always find clever ways to do things. I am rarely quiet. I feel a great deal of sympathy for others. I am very systematic almost always keep things in order. I am sometimes tense. I am not at all fascinated by art music or literature. I am very dominant almost always act as a leader. I often start arguments with others. I have some difficulty getting started on tasks. I feel very secure comfortable wit

In [3]:
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)
completion = client.chat.completions.create(
    temperature=1,
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": "You are a person with the given personality traits"},
        {"role": "user", "content": prompt}
    ]
)
completion.choices[0].message.content

'```json\n{\n  "Investment": 7,\n  "Extreme_Sports": 3,\n  "Entrepreneurial_Venture": 2,\n  "Confessing_Feelings": 5,\n  "Study_Overseas": 1\n}\n```'

# Test with multiple participants

In [9]:
import pandas as pd
from openai import OpenAI
from risk_taking import get_prompt
import os
import json
import re
from pathlib import Path
import concurrent.futures
import time
from tenacity import retry, stop_after_attempt, wait_exponential

# Load data
df = pd.read_csv('../data/york_data_clean.csv')

client = OpenAI(api_key=api_key)

# Set temperature as a constant
TEMPERATURE = 0

def extract_jsons(text):
    json_pattern = r'(```json\s*)?\s*(\{[^}]+\})\s*(```)?'
    matches = re.finditer(json_pattern, text, re.DOTALL)
    extracted_jsons = []
    for match in matches:
        json_str = match.group(2)
        try:
            json_obj = json.loads(json_str)
            extracted_jsons.append(json_obj)
        except json.JSONDecodeError:
            print(f"Warning: Could not parse JSON: {json_str}")
    return extracted_jsons

@retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=4, max=10))
def process_participant(person):
    prompt = get_prompt(person['bfi_combined'])
    
    completion = client.chat.completions.create(
        temperature=TEMPERATURE,
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are a person with the given personality traits. Respond with a JSON object containing your risk assessment."},
            {"role": "user", "content": prompt}
        ]
    )
    
    response_text = completion.choices[0].message.content
    extracted_jsons = extract_jsons(response_text)
    
    if extracted_jsons:
        return extracted_jsons[0]  # Return the first extracted JSON
    else:
        raise ValueError("No valid JSON found in the response")

def process_participant_with_retries(index):
    person = df.iloc[index]
    try:
        response = process_participant(person)
        return index, response
    except Exception as e:
        print(f"Error processing participant {index}: {str(e)}")
        return index, {"error": str(e)}

# Process all participants in parallel
num_participants = len(df)
results = [None] * num_participants

# Process participants in batches to manage memory and API rate limits
BATCH_SIZE = 50

for batch_start in range(0, num_participants, BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, num_participants)
    print(f"Processing participants {batch_start} to {batch_end - 1}")

    with concurrent.futures.ThreadPoolExecutor(max_workers=BATCH_SIZE) as executor:
        future_to_index = {executor.submit(process_participant_with_retries, i): i for i in range(batch_start, batch_end)}
        
        for future in concurrent.futures.as_completed(future_to_index):
            index, result = future.result()
            results[index] = result

    print(f"Completed batch {batch_start} to {batch_end - 1}")

# Create a 'risk' directory in the current working directory
risk_dir = Path.cwd() / "risk"
risk_dir.mkdir(exist_ok=True)

# Save results to JSON file with temperature in the filename
output_file = risk_dir / f"risk_{TEMPERATURE}.json"
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"Data processed and saved to {output_file}")

Processing participants 0 to 49
Completed batch 0 to 49
Processing participants 50 to 99
Completed batch 50 to 99
Processing participants 100 to 149
Completed batch 100 to 149
Processing participants 150 to 199
Completed batch 150 to 199
Processing participants 200 to 249
Completed batch 200 to 249
Processing participants 250 to 299
Completed batch 250 to 299
Processing participants 300 to 349
Completed batch 300 to 349
Processing participants 350 to 355
Completed batch 350 to 355
Data processed and saved to /Users/mhhuang/psychometrics_AI/human_study/simulation/risk/risk_0.8.json
